In [265]:
# CS 124: Machine Learning in Genetics
# Project: Haplotype Phaser
# Contributors: Aditya Pimplaskar, Aditya Joglekar
import numpy as np
import pandas as pd
from sklearn.impute import KNNImputer
from sklearn.impute import SimpleImputer

In [269]:
# Imputation functions
def fillna(col):
    col.fillna(col.value_counts().index[0], inplace=True)
    return col

def imputeData(df):
    df = df.replace('*', np.NaN)
    #imputer = SimpleImputer(missing_values=np.nan, strategy= 'most_frequent')
    #return pd.DataFrame(imputer.fit_transform(df))
    return df.apply(lambda col:fillna(col))


def deleteDups(g):
    # goal is to eliminate duplicate genotypes/duplicate haplotypes
    g = g.drop_duplicates(inplace=True)
    return g

In [187]:
# Compatibility checker function
def checkPhase(g, h1, h2):
    # want to see if element wise sum of h1 and h2 is g
    import numpy as np
    g = np.array(g)
    h1 = np.array(h1)
    h2 = np.array(h2)
    comparison =  (h1 + h2 == g)
    return comparison.all()

def findDifference(g, h1): 
    # so that we can fill in a new haplo if we don't find a compatibile pair in Clark's
    g = np.array(g)
    h1 = np.array(h1)
    return g - h1


In [196]:
def clarks(genotypes):
    # need to give a starting pool
    # we can do this by phasing all of the deterministic genotypes
    
    haplotypes = []
    
    toDrop = [] # deterministic
    for ind in range(len(genotypes)):
        h = []
        g = genotypes.iloc[ind]
        for i in g:
            if i == 1: #non deterministic
                break
            if i == 0:
                h.append(0)
            if i == 2:
                h.append(1)
        if len(h) == len(g): # did you make it to the end of the string
            haplotypes.append(h)
            toDrop.append(ind) #thins out our new genotype list
    genotypes = genotypes.drop(genotypes.index[toDrop])
    
    for i,g in genotypes.iterrows():
        phased = False #flag variable
        for h1 in range(len(haplotypes)):
            for h2 in range(h1, len(haplotypes)):
                if checkPhase(g,haplotypes[h1],haplotypes[h2]): # we already have the phase accounted for
                    phased = True
        if phased == False: # now we need to add a haplo that works
            for h in haplotypes:
                diff = findDifference(g, h)
                # now just need to make sure this difference has no weird values -- i.e. is a valid addition
                if sum(0 <= x <= 1 for x in diff) == len(g):
                    haplotypes.append(h)
                    break
    
    return haplotypes

In [215]:
def wrapper(genotypes):
    # do it all
    genotypes = genotypes.T
    genotypes = imputeData(genotypes) # impute
    deleteDups(genotypes) # get rid of duplicates
    return clarks(genotypes)

In [275]:
ex1 = pd.read_csv("assignment/example_data_1_masked.txt", sep = " ", header=None)
ex1 = ex1.head(100)
ex1

0  1  2  3  4  5  6  7  8  9  ... 40 41 42 43 44 45 46 47 48 49
0   0  1  2  2  1  1  *  2  1  2 ...  2  1  *  1  1  0  2  0  2  2
1   *  2  1  1  1  2  1  2  2  2 ...  1  1  2  2  2  2  2  2  1  1
2   *  2  2  1  2  1  2  2  2  1 ...  2  1  2  2  2  1  2  2  2  1
3   *  2  2  *  2  2  2  2  2  2 ...  2  2  *  2  2  2  *  2  2  2
4   2  *  2  2  2  2  2  *  2  2 ...  2  *  2  *  2  2  2  2  2  2
5   0  *  0  0  0  0  *  0  *  * ...  0  0  0  0  0  0  0  0  0  0
6   0  0  0  0  0  0  0  0  0  0 ...  0  0  0  0  0  0  0  0  0  0
7   2  2  1  2  2  *  2  2  2  2 ...  *  2  2  2  2  2  2  2  *  2
8   2  2  *  2  2  2  2  2  2  2 ...  2  2  2  2  2  2  2  2  2  2
9   2  0  0  0  0  1  0  1  0  0 ...  0  0  1  0  0  0  0  1  *  0
10  0  1  0  0  0  0  0  0  0  0 ...  0  0  0  1  0  1  0  *  0  0
11  2  1  *  2  2  2  2  2  2  2 ...  2  2  2  1  2  1  2  2  2  2
12  0  0  0  0  *  0  0  0  0  * ...  0  0  0  0  0  1  0  0  *  0
13  0  1  1  0  1  0  0  0  0  0 ...  0  0  0  *  0  1  1  0  0  0
14  *  2  2  2  2  2  2  2  2  2 ...  2  *  2  2  2  2  2  2  2  2
15  *  0  0  0  0  0  0  0  0  0 ...  0  0  0  1  0  0  0  *  *  0
16  *  0  0  0  0  1  0  0  *  0 ...  0  0  0  0  0  0  0  0  0  0
17  0  1  0  0  0  0  0  0  0  * ...  1  0  *  0  1  0  0  0  *  1
18  0  *  1  *  1  0  0  0  0  0 ...  0  *  0  0  0  1  1  0  0  0
19  2  2  2  2  1  2  1  2  2  2 ...  *  2  2  2  2  2  2  2  2  2
20  1  2  1  2  2  2  2  2  2  1 ...  1  2  2  2  2  1  2  1  2  1
21  0  0  0  0  0  0  1  1  0  0 ...  1  1  *  0  *  0  0  *  1  *
22  2  1  2  2  0  2  1  2  2  2 ...  1  2  1  1  *  1  1  2  *  1
23  0  0  0  0  *  0  0  0  0  0 ...  0  0  0  0  *  0  0  0  0  0
24  2  2  2  2  2  2  2  2  2  2 ...  2  2  2  *  2  2  2  2  2  2
25  2  2  2  2  *  2  2  2  2  2 ...  2  2  2  *  2  2  2  2  2  2
26  2  2  2  2  2  2  2  2  2  * ...  2  2  2  1  2  2  2  2  2  2
27  0  0  0  1  0  1  0  0  0  0 ...  0  0  0  0  *  0  0  0  *  *
28  2  *  1  1  2  2  2  2  2  2 ...  2  2  2  2  2  *  2  2  2  2
29  *  1  *  0  0  *  0  0  0  0 ...  0  0  0  0  0  0  0  0  *  0
.. .. .. .. .. .. .. .. .. .. .. ... .. .. .. .. .. .. .. .. .. ..
70  0  0  1  0  0  0  1  0  1  0 ...  1  0  *  0  0  0  1  1  1  0
71  0  *  1  0  0  0  0  0  0  0 ...  0  0  0  0  0  0  1  1  1  0
72  0  0  0  0  0  0  *  0  0  0 ...  0  0  0  0  0  0  0  0  0  0
73  2  2  1  2  2  2  1  2  2  2 ...  1  2  2  2  2  2  1  1  1  2
74  0  0  *  0  0  0  0  0  0  0 ...  *  *  0  0  0  0  0  *  *  *
75  0  0  *  1  2  0  *  1  0  2 ...  0  1  1  1  1  *  1  1  1  1
76  2  2  2  2  2  2  2  2  2  2 ...  2  2  2  2  2  2  2  2  2  2
77  0  0  *  1  0  0  0  0  0  0 ...  0  0  *  0  0  0  0  0  0  0
78  2  *  2  1  *  2  2  1  2  2 ...  2  2  2  2  2  2  2  2  2  2
79  2  2  1  2  2  2  2  2  2  2 ...  2  2  *  2  2  2  1  2  1  2
80  2  2  2  2  2  2  2  1  1  2 ...  2  2  2  2  *  2  2  2  2  2
81  *  *  1  0  0  0  *  0  0  0 ...  0  0  0  0  0  0  *  1  1  0
82  0  0  0  0  0  0  0  0  0  0 ...  0  0  *  0  0  0  1  0  0  1
83  2  2  2  2  *  2  2  *  2  2 ...  2  2  2  2  2  2  2  2  2  2
84  2  2  2  2  2  2  *  2  2  2 ...  2  2  1  2  2  2  *  2  2  2
85  1  1  0  2  1  1  0  *  2  0 ...  1  1  2  0  0  0  0  0  0  0
86  0  *  1  *  0  2  1  1  2  1 ...  2  *  1  0  *  0  1  1  1  1
87  2  2  2  1  *  *  1  2  2  * ...  1  *  2  1  1  2  1  1  1  0
88  0  *  0  0  0  0  0  0  0  0 ...  0  0  0  0  0  0  0  *  0  0
89  0  1  1  1  2  1  1  2  1  0 ...  0  1  1  1  1  2  1  1  1  0
90  2  1  1  2  1  2  1  1  1  2 ...  2  1  1  1  1  0  1  1  1  2
91  1  2  2  2  2  2  1  2  2  0 ...  1  2  2  1  1  2  1  *  1  1
92  1  1  1  2  2  1  2  2  2  2 ...  1  1  0  *  2  2  2  2  2  2
93  1  2  2  2  2  2  2  2  2  1 ...  1  2  2  2  2  2  2  1  1  1
94  *  1  1  0  *  0  0  0  *  * ...  2  1  1  1  1  *  1  1  1  2
95  0  0  0  0  0  0  0  0  0  0 ...  0  *  0  0  0  0  0  0  0  0
96  0  1  1  1  2  2  2  2  2  0 ...  0  1  0  1  1  2  1  1  1  0
97  2  1  1  1  0  0  0  0  0  2 ...  2  1  2  1  

In [285]:
ex1_imp = imputeData(ex1.T)
sum(x == 0 or x == 2 for x in ex1_imp)

2